In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_bootstrap import Bootstrap
from wtforms import StringField,PasswordField,SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Length, ValidationError, Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime
import pandas as pd
import sqlite3

app = Flask(__name__)

bootstrap = Bootstrap(app)
bcrypt = Bcrypt (app)
app.config['SQLALCHEMY_DATABASE_URI']= 'sqlite:///user.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)
app.config['SECRET_KEY']= 'thisisasecretkey'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

class Note(db.Model):
    __tablename__ = 'note'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e)  
        
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    notes = db.relationship('Note', backref='writer', lazy='dynamic')
try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e)  
    
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

    
class SignupForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(),Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")
            
class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login") 

class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    note_body = StringField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Completion Date"})
    submit = SubmitField("Add Note")    
    
@app.route('/', methods=['POST', 'GET'])
def index():
        return render_template('index.html')

@app.route('/newnote', methods=['GET','POST'])
@login_required
def newnote():
    form = NewNoteForm()
    if form.validate_on_submit():
        newnote = Note(title=form.title.data, note_body=form.note_body.data, writer=current_user)
        db.session.add(newnote)
        db.session.commit()
        return redirect(url_for('mynotes'))
    return render_template('newnote.html', title='New Note', form=form)

@app.route('/mynotes', methods=['GET','POST'])
@login_required
def mynotes():
    notes = Note.query.all()
    return render_template('mynotes.html', notes=notes, title='My Notes')

@app.route('/delete_note/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('mynotes'))

@app.route('/update/<int:note_id>', methods=['GET', 'POST'])
@login_required
def update_note(note_id):
    note = Note.query.get_or_404(note_id)

    if request.method == 'POST':
        note.title = request.form.get('Title')
        note.note_body = request.form.get('Note Body')
        db.session.commit()
        return redirect(url_for('mynotes'))
    return render_template('update.html', note=note, title='My Notes')


@app.route("/transportation")
def transportation():
    attractions = [{'Location': 'Tian Tan Buddha (Big Buddha)', 'District': 'New Territories', 'More Info': 'https://www.tripadvisor.com.hk/Attraction_Review-g294217-d537975-Reviews-Tian_Tan_Buddha_Big_Buddha-Hong_Kong.html'},
        {'Location': 'Victoria Peak (The Peak)',  'District': 'Causeway Bay', 'More Info': 'https://www.tripadvisor.com.hk/Attraction_Review-g294217-d311573-Reviews-Victoria_Peak_The_Peak-Hong_Kong.html'},
        {'Location': 'Star Ferry',  'District': 'Tsim Sha Tsui',  'More Info': 'https://www.tripadvisor.com.hk/Attraction_Review-g294217-d2228870-Reviews-Star_Ferry-Hong_Kong.html'}] 
    df = pd.DataFrame(attractions)
    df.to_csv('attractions.csv',mode='w',index=False)
    df = pd.read_csv("attractions.csv")
    list = df.to_dict('records')
    return render_template('transportation.html', entries = attractions)

@app.route("/tips")
def tips():
    return render_template('tips.html')

@app.route("/about us")
def about_us():
    return render_template('about us.html')

@app.route("/categories")
def categories():
    return render_template('categories.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("newnote"))

        return '<h1>Invalid username or password</h1>'
    return render_template('login.html', form=form)                             
                    
@app.route('/logout', methods=["GET","POST"])
def logout():
    logout_user()
    return redirect(url_for('login'))

@app.route("/signup", methods=["GET", "POST"])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        hashed_password = generate_password_hash(form.password.data, method='sha256')
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('signup.html', title='Signup', form=form)

if __name__ == '__main__':
    app.run('localhost', 9005,debug=False)
#username is admin and password is abcd

Database successfully created.
Database successfully created.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9005/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Oct/2021 02:15:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:15:34] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:15:38] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [29/Oct/2021 02:15:38] "GET /newnote HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:15:45] "GET /mynotes HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:15:48] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [29/Oct/2021 02:15:48] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:15:52] "GET /signup HTTP/1.1" 200 -
